In [34]:
import os
import finnhub
import datetime
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.core_plugins.time_plugin import TimePlugin
from semantic_kernel.core_plugins.math_plugin import MathPlugin
from semantic_kernel.core_plugins.text_plugin import TextPlugin
from semantic_kernel.prompt_template import PromptTemplateConfig


class FinnhubPlugin:
    def __init__(self, api_key):
        self.client = finnhub.Client(api_key=api_key)

    def fetch_latest_investment_news(self, category="top news", min_id=0):
        try:
            news = self.client.general_news(category, min_id=min_id)
            formatted_news = []
            for item in news:
                formatted_news.append(
                    {
                        "headline": item["headline"],
                        "source": item["source"],
                        "date": datetime.datetime.fromtimestamp(
                            item["datetime"]
                        ).strftime("%Y-%m-%d %H:%M:%S"),
                        "summary": item["summary"],
                        "url": item["url"],
                        "image": item["image"],
                    }
                )
            return formatted_news
        except Exception as e:
            return {"error": str(e)}


# API Keys from environment variables
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
FINNHUB_API_KEY = os.environ["FINNHUB_API_KEY"]

# Initialize Finnhub plugin
finnhub_plugin = FinnhubPlugin(FINNHUB_API_KEY)

# Initialize the Semantic Kernel
kernel = Kernel()

# Add OpenAI services
gpt4o_service_id = "chat-gpt-4o"
kernel.add_service(
    OpenAIChatCompletion(
        service_id=gpt4o_service_id, api_key=OPENAI_API_KEY, ai_model_id="gpt-4o"
    )
)

gpt35_service_id = "chat-gpt-35"
kernel.add_service(
    OpenAIChatCompletion(
        service_id=gpt35_service_id, api_key=OPENAI_API_KEY, ai_model_id="gpt-3.5-turbo"
    )
)

# Add core plugins
kernel.add_plugin(TimePlugin(), "time")
kernel.add_plugin(MathPlugin(), "math")
kernel.add_plugin(TextPlugin(), "text")

# Add Finnhub plugin to kernel
kernel.add_plugin(finnhub_plugin, "finnhub")

# Define the request settings
req_settings = kernel.get_prompt_execution_settings_from_service_id(gpt35_service_id)
req_settings.max_tokens = 2000
req_settings.temperature = 0.7
req_settings.top_p = 0.8

# Define the prompt template for summarizing investment news
investment_news_prompt_template = """
Summarize 5 top news bullet points. News: {{$news}}
"""

# Create the prompt template configuration
investment_news_prompt_template_config = PromptTemplateConfig(
    template=investment_news_prompt_template,
    name="investment_news_summary",
    template_format="semantic-kernel",
    execution_settings=req_settings,
)

# Create the function using the prompt template configuration
investment_news_summary_function = kernel.add_function(
    function_name="investment_news_summary_function",
    plugin_name="investment_news_plugin",
    prompt_template_config=investment_news_prompt_template_config,
)

news_summary = await kernel.invoke(
    investment_news_summary_function, news=finnhub_plugin.fetch_latest_investment_news()
)

In [33]:
news_summary

FunctionResult(function=KernelFunctionMetadata(name='investment_news_summary_function', plugin_name='investment_news_plugin', description=None, parameters=[KernelParameterMetadata(name='news', description='', default_value='', type_='', is_required=True, type_object=None, schema_data={'type': 'object'})], is_prompt=True, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='The completion result', default_value=None, type_='FunctionResult', is_required=True, type_object=None, schema_data=None), additional_properties=None), value=[ChatMessageContent(inner_content=ChatCompletion(id='chatcmpl-9UlDxFjHo0dzUeSpuAKJ1GElba19U', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="1. Former President Donald Trump has been found dead in his apartment.\n2. The cause of Trump's death has not been disclosed.\n3. Trump's death has shocked and saddened many across the country.\n4. Tributes and condolences are pourin

In [26]:
# Define the prompt template for summarizing
summarize_prompt_template = """
{{$input}}

One line TLDR with the fewest words. Last time you said: {{$history}}. Explain also what day it is Context: {{time.today}}.
"""

# Create the prompt template configuration
summarize_prompt_template_config = PromptTemplateConfig(
    template=summarize_prompt_template,
    name="summarize",
    template_format="semantic-kernel",
    execution_settings=req_settings,
)

# Create the summarize function using the prompt template configuration
summarize = kernel.add_function(
    function_name="summarize_function",
    plugin_name="summarize_plugin",
    prompt_template_config=summarize_prompt_template_config,
)


# Run your prompt
# Note: functions are run asynchronously
async def main():
    # Summarize the laws of thermodynamics
    thermodynamics_result = await kernel.invoke(
        summarize,
        input="""
    1st Law of Thermodynamics - Energy cannot be created or destroyed.
    2nd Law of Thermodynamics - For a spontaneous process, the entropy of the universe increases.
    3rd Law of Thermodynamics - A perfect crystal at zero Kelvin has zero entropy.""",
        history="",
    )
    print(thermodynamics_result)

In [27]:
await main()

1st Law: Energy is conserved.
2nd Law: Entropy increases in spontaneous processes.
3rd Law: Zero entropy at absolute zero.
Today is Thursday, 30 May, 2024.


In [ ]:
# setup language models
gpt4_kernel = sk.Kernel()
gpt4_kernel.add_service(
    OpenAIChatCompletion(
        service_id="gpt4o",
        ai_model_id="gpt-4o",
        api_key=OPENAI_API_KEY
    ),
)
gpt3_kernel = sk.Kernel()
gpt3_kernel.add_service(
    OpenAIChatCompletion(
        service_id="gpt3.5-turbo",
        ai_model_id="gpt-3.5-turbo",
        api_key=OPENAI_API_KEY
    ),
)

# examples of adding functions to the kernels
gpt4_kernel.add_function(function_name="get_stock_price", plugin_name="stock_info_plugin", prompt="What is the stock price of {symbol}?")
gpt4_kernel.add_function(function_name="get_company_profile", plugin_name="stock_info_plugin", prompt="Tell me about the company {symbol}.")
gpt4_kernel.add_function(function_name="get_historical_data", plugin_name="stock_info_plugin", prompt="Give me the historical data for {symbol}.")
gpt3_kernel.add_function(function_name="get_stock_price", plugin_name="stock_info_plugin", prompt="What is the stock price of {symbol}?")
gpt3_kernel.add_function(function_name="get_company_profile", plugin_name="stock_info_plugin", prompt="Tell me about the company {symbol}.")
gpt3_kernel.add_function(function_name="get_historical_data", plugin_name="stock_info_plugin", prompt="Give me the historical data for {symbol}.")


# for using Phi-3 locally
qa_pipeline = pipeline('text-generation', model='microsoft/Phi-3-mini-128k-instruct')

In [ ]:
test_function = gpt3_kernel.get_function(function_name="get_stock_price", plugin_name="stock_info_plugin")
request_settings = {
    "prompt": "What is the stock price of AAPL?",
    "max_tokens": 150,  # Adjust as needed
    "temperature": 0.7,
    "top_p": 0.9
}

# Invoke the function
result = await gpt3_kernel.invoke(test_function, **request_settings)
print(result.get_inner_content().to_dict()['choices'][0]['message']['content'])

In [ ]:
finnhub_client = finnhub.Client(FINNHUB_API_KEY)
trading_client = TradingClient(ALPACA_API_KEY, ALPACA_SECRET_KEY, paper=True)

# Initialize FAISS index
d = 512  # Example dimensionality
index = faiss.IndexIVFPQ(faiss.IndexFlatL2(d), d, 256, 8, 8)
symbol_mapping = {}

def fetch_stock_symbols():
    #TODO: Fetch stock symbols from API
    return ["AAPL", "MSFT", "AMZN", "GOOGL", "FB", "JPM", "JNJ", "NVDA", "PG", "DIS"]

def fetch_stock_data(symbol):
    # Example function to fetch stock data
    url = f"https://api.example.com/stock/{symbol}/fundamentals"
    response = requests.get(url)
    data = response.json()
    return data

def preprocess_data(data):
    # Convert your data to a vector
    # Ensure the order of features matches the dimensionality 'd'
    features = ['open', 'high', 'low', 'close', 'volume']
    vector = np.array([data[feature] for feature in features]).astype('float32')
    return vector

def store_data_in_faiss(symbol, data):
    vector = preprocess_data(data)
    index.add(np.array([vector]))
    symbol_mapping[index.ntotal - 1] = symbol

def fetch_latest_data_from_faiss():
    # Example function to fetch latest data from FAISS
    k = 1  # Number of nearest neighbors
    query_vector = np.random.random(d).astype('float32')  # Example query vector
    D, I = index.search(np.array([query_vector]), k)
    symbols = [symbol_mapping[i] for i in I[0]]
    return symbols

def create_debate_prompt(stock_data):
    prompt = [
        {"role": "system", "content": "You are a financial expert."},
        {"role": "user", "content": f"Analyze the following stock data and debate: {stock_data}"}
    ]
    return prompt

def debate_agents(stock_data):
    prompt = create_debate_prompt(stock_data)
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=prompt,
        temperature=0.7
    )
    return parse_debate_response(response)

def parse_debate_response(response):
    decisions = [message['content'] for message in response['choices']]
    decision_counts = {"buy": 0, "sell": 0, "hold": 0}
    for decision in decisions:
        if "buy" in decision:
            decision_counts["buy"] += 1
        elif "sell" in decision:
            decision_counts["sell"] += 1
        else:
            decision_counts["hold"] += 1
    return max(decision_counts, key=decision_counts.get)

def execute_trade(decision, symbol):
    if decision == 'buy':
        order = MarketOrderRequest(
            symbol=symbol,
            qty=1,
            side=OrderSide.BUY,
            time_in_force=TimeInForce.GTC
        )
    elif decision == 'sell':
        order = MarketOrderRequest(
            symbol=symbol,
            qty=1,
            side=OrderSide.SELL,
            time_in_force=TimeInForce.GTC
        )
    
    trading_client.submit_order(order)

In [ ]:
# List of stock symbols to evaluate
stocks = ['AAPL', 'MSFT', 'GOOGL']

def get_stock_fundamentals(stock_symbol):
    """Fetches basic fundamental data for a given stock symbol using Finnhub API."""
    try:
        # Get basic financials
        financials = finnhub_client.company_basic_financials(stock_symbol, 'all')

        # Extract relevant data
        metric = financials['metric']
        fundamentals = {
            'symbol': stock_symbol,
            'peRatioTTM': metric.get('peNormalizedAnnual', 'N/A'),
            'pbRatio': metric.get('pbAnnual', 'N/A'),
            'dividendYieldTTM': metric.get('currentDividendYieldTTM', 'N/A'),
            'epsTTM': metric.get('epsTTM', 'N/A'),
            'marketCapitalization': metric.get('marketCapitalization', 'N/A'),
            '52WeekHigh': metric.get('52WeekHigh', 'N/A'),
            '52WeekLow': metric.get('52WeekLow', 'N/A'),
        }
        return fundamentals
    except Exception as e:
        print(f"Error fetching data for {stock_symbol}: {e}")
        return None

def evaluate_stocks(stock_list):
    """Evaluates a list of stocks and prints their fundamental data."""
    results = []
    for stock in stock_list:
        fundamentals = get_stock_fundamentals(stock)
        if fundamentals:
            results.append(fundamentals)
    
    return results

import finnhub
import os

# List of stock symbols to evaluate
stocks = ['AAPL', 'MSFT', 'GOOGL']

def get_stock_fundamentals(stock_symbol):
    """Fetches basic fundamental data for a given stock symbol using Finnhub API."""
    try:
        # Get basic financials
        financials = finnhub_client.company_basic_financials(stock_symbol, 'all')

        # Get current price
        quote = finnhub_client.quote(stock_symbol)
        current_price = quote['c']

        # Extract relevant data
        metric = financials['metric']
        fundamentals = {
            'symbol': stock_symbol,
            'current_price': current_price,
            '10DayAverageTradingVolume': metric.get('10DayAverageTradingVolume', 'N/A'),
            '13WeekPriceReturnDaily': metric.get('13WeekPriceReturnDaily', 'N/A'),
            '26WeekPriceReturnDaily': metric.get('26WeekPriceReturnDaily', 'N/A'),
            '52WeekHigh': metric.get('52WeekHigh', 'N/A'),
            '52WeekLow': metric.get('52WeekLow', 'N/A'),
            '52WeekPriceReturnDaily': metric.get('52WeekPriceReturnDaily', 'N/A'),
            '5DayPriceReturnDaily': metric.get('5DayPriceReturnDaily', 'N/A'),
            'peRatioTTM': metric.get('peNormalizedAnnual', 'N/A'),
            'pbRatio': metric.get('pbAnnual', 'N/A'),
            'dividendYieldTTM': metric.get('currentDividendYieldTTM', 'N/A'),
            'epsTTM': metric.get('epsTTM', 'N/A'),
            'marketCapitalization': metric.get('marketCapitalization', 'N/A'),
            'currentRatioAnnual': metric.get('currentRatioAnnual', 'N/A'),
            'quickRatioAnnual': metric.get('quickRatioAnnual', 'N/A'),
            'debt/equityAnnual': metric.get('totalDebt/totalEquityAnnual', 'N/A'),
            'grossMarginAnnual': metric.get('grossMarginAnnual', 'N/A'),
            'netProfitMarginAnnual': metric.get('netProfitMarginAnnual', 'N/A'),
            'operatingMarginAnnual': metric.get('operatingMarginAnnual', 'N/A'),
            'revenueGrowth3Y': metric.get('revenueGrowth3Y', 'N/A'),
            'revenueGrowth5Y': metric.get('revenueGrowth5Y', 'N/A'),
            'revenueGrowthQuarterlyYoy': metric.get('revenueGrowthQuarterlyYoy', 'N/A'),
            'epsGrowth3Y': metric.get('epsGrowth3Y', 'N/A'),
            'epsGrowth5Y': metric.get('epsGrowth5Y', 'N/A'),
            'bookValuePerShareAnnual': metric.get('bookValuePerShareAnnual', 'N/A'),
            'cashFlowPerShareTTM': metric.get('cashFlowPerShareTTM', 'N/A'),
            'enterpriseValue': metric.get('enterpriseValue', 'N/A'),
        }
        return fundamentals
    except Exception as e:
        print(f"Error fetching data for {stock_symbol}: {e}")
        return None

def evaluate_stocks(stock_list):
    """Evaluates a list of stocks and prints their fundamental data."""
    results = []
    for stock in stock_list:
        fundamentals = get_stock_fundamentals(stock)
        if fundamentals:
            results.append(fundamentals)
    
    return results

if __name__ == "__main__":
    stock_fundamentals = evaluate_stocks(stocks)
    for stock in stock_fundamentals:
        print(stock)

print(pd.DataFrame(stock_fundamentals))

In [ ]:
import pandas as pd
pd.DataFrame(finnhub_client.company_basic_financials('MSFT', 'All')).T.to_json()

In [ ]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest

trading_client = TradingClient(ALPACA_API_KEY, ALPACA_SECRET_KEY, paper=False)

# Get our account information.
account = trading_client.get_account()

# Check if our account is restricted from trading.
if account.trading_blocked:
    print('Account is currently restricted from trading.')

# Check how much money we can use to open new positions.
print('${} is available as buying power.'.format(account.buying_power))

In [ ]:
trading_client.get_all_positions()

In [ ]:
symbol = "MSFT"  # replace with your symbol
url = f"https://data.alpaca.markets/v2/stocks/{symbol}/quotes/latest"

headers = {
    'accept': 'application/json',
    'APCA-API-KEY-ID': ALPACA_API_KEY,
    'APCA-API-SECRET-KEY': ALPACA_SECRET_KEY
}

response = requests.get(url, headers=headers)
data = response.json()
print(data)

In [ ]:
url = "https://data.alpaca.markets/v2/positions"
headers = {
    'accept': 'application/json',
    'APCA-API-KEY-ID': ALPACA_API_KEY,
    'APCA-API-SECRET-KEY': ALPACA_SECRET_KEY
}

response = requests.get(url, headers=headers)
data = response.json()
print(data)